<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/5_rgb_xyz_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-14 21:11:16--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11733 (11K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.46K  --.-KB/s    in 0s      

2022-06-14 21:11:16 (63.2 MB/s) - ‘utils.py’ saved [11733/11733]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import ModelCreator, get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 1.6 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [49]:
train_ds, _ = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3510 files belonging to 16 classes.


In [33]:
classes, class_weight = get_class_weight()

In [50]:
def combine_rgb_xyz(alpha):
  def combiner(img_batch):
    img_batch_xyz = tfio.experimental.color.rgb_to_xyz(img_batch)
    return img_batch*alpha + img_batch_xyz*(1-alpha)
  return combiner

In [51]:
combine_rgb_xyz = combine_rgb_xyz(.6)

train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (combine_rgb_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [52]:
rgb_xyz_creator = ModelCreator(HUB_URL, 'RGB_XYZ-model')
rgb_xyz_model = rgb_xyz_creator.make_model(img_size=IMG_SIZE)

Model: "RGB_XYZ-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_6 (KerasLayer)  (None, 2048)              23500352  
                                                                 
 dense_6 (Dense)             (None, 16)                32784     
                                                                 
Total params: 23,533,136
Trainable params: 32,784
Non-trainable params: 23,500,352
_________________________________________________________________


In [36]:
callbacks = rgb_xyz_creator.get_callbacks()

In [53]:
history = rgb_xyz_model.fit(train_ds,
                            callbacks=callbacks,
                            epochs=25,
                            class_weight=class_weight)

Epoch 1/7
519/519 [==============================] - 197s 353ms/step - loss: 123.6843 - auc: 0.6291 - accuracy: 0.6905
Epoch 2/7
519/519 [==============================] - 183s 349ms/step - loss: 77.5934 - auc: 0.7019 - accuracy: 0.7700
Epoch 3/7
519/519 [==============================] - 184s 352ms/step - loss: 68.1623 - auc: 0.7352 - accuracy: 0.7966
Epoch 4/7
519/519 [==============================] - 183s 350ms/step - loss: 55.1114 - auc: 0.7732 - accuracy: 0.8259
Epoch 5/7
519/519 [==============================] - 181s 347ms/step - loss: 45.7943 - auc: 0.7982 - accuracy: 0.8421
Epoch 6/7
519/519 [==============================] - 181s 346ms/step - loss: 48.1038 - auc: 0.8042 - accuracy: 0.8483
Epoch 7/7
519/519 [==============================] - 181s 346ms/step - loss: 36.7506 - auc: 0.8276 - accuracy: 0.8653
